**Capstone 2: Pre-Processing**

Problem Statement: The National Transportation Safety Board (NTSB) has an Aviation Accident Database that contains information about civil aviation accidents and incidents that they investigate within the United States, its territories and international waters. For this Project, we will specifically be looking at airplane accidents for the 20 years from 2002 to 2021. The operations included are scheduled Part 121: Air Carrier, which includes most passenger airlines, as well as Part 135: Small Carrier for both commuter and ‘air taxi’ small aircraft. This dataset does not include aircraft accidents where the aircraft involved was amateur built.

**Notebook Objectives:**


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
df = pd.read_excel('data/flights_EDA.xlsx')
df.head()

,NtsbNo,EventType,Mkey,EventDate,City,State,Country,N#,HasSafetyRec,Mode,...,FAR Part,Findings_supp,Findings_main1,Findings_main2,Findings_main3,Findings_main4,Findings_main5,Findings_main6,Region,SecondaryCause
0,ANC08MA038,ACC,67341,2008-01-05T13:43:00Z,Kodiak,Alaska,United States,N509FN,False,Aviation,...,135,Personnel issues - Task performance - Mainten...,Aircraft,Aircraft structures,Doors,Cargo/baggage doors,Incorrect service/maintenance,NaN,Pacific West,Air_Maintenance Issue
1,ANC08LA031,ACC,67395,2008-01-15T14:39:00Z,Bethel,Alaska,United States,N454SF,False,Aviation,...,135,Personnel issues - Action/decision - Info pro...,Environmental issues,Physical environment,Runway/land/takeoff/taxi surface,Snow/slush/ice covered surface,Not specified,NaN,Pacific West,Env_Runway
2,ANC08LA046,ACC,67751,2008-04-01T14:40:00Z,Koliganek,Alaska,United States,N8327S,False,Aviation,...,135,Environmental issues - Physical environment -...,Environmental issues,Conditions/weather/phenomena,Light condition,Flat light,Effect on personnel,NaN,Pacific West,Env_Lighting/Visibility
3,ANC08FA050,ACC,67801,2008-04-09T15:30:00Z,Unalaska,Alaska,United States,N741,False,Aviation,...,135,Personnel issues - Action/decision - Info pro...,Environmental issues,Physical environment,Object/animal/substance,Ground vehicle,Contributed to outcome,NaN,Pacific West,Env_Object/Animal
4,ANC08CA065,ACC,68340,2008-04-28T13:30:00Z,Talkeetna,Alaska,United States,N565TA,False,Aviation,...,135,Personnel issues - Action/decision - Info pro...,Environmental issues,Physical environment,Runway/land/takeoff/taxi surface,Snow/slush/ice covered surface,Not specified,NaN,Pacific West,Env_Runway


In [30]:
df.columns

Index(['NtsbNo', 'EventType', 'Mkey', 'EventDate', 'City', 'State', 'Country',
       'N#', 'HasSafetyRec', 'Mode', 'OriginalPublishedDate',
       'HighestInjuryLevel', 'FatalInjuryCount', 'SeriousInjuryCount',
       'MinorInjuryCount', 'ProbableCause', 'FAR Part', 'Findings_supp',
       'Findings_main1', 'Findings_main2', 'Findings_main3', 'Findings_main4',
       'Findings_main5', 'Findings_main6', 'Region', 'SecondaryCause'],
      dtype='object')

I'm going to delete some columns since I'm about to make a lot of new ones with one-hot encoding the categorical data. I've narrowed down what I'm looking at overall: where the accident occured (region) and the main and secondary causes.

Columns to be deleted: EventType (all ACC), Mode (all aviation), originalPublishedDate (date the report was published), ProbableCause (covered by main and secondary causes), Findings_supp, and all Findings_main except main1.

In [31]:
df = df.drop(['EventType', 'Mode', 'OriginalPublishedDate', 'ProbableCause', 'Findings_supp'], axis = 1)
df = df.drop(['Findings_main2', 'Findings_main3', 'Findings_main4', 'Findings_main5', 'Findings_main6'], axis = 1)
df.rename(columns = {'Findings_main1':'Failure_Category'}, inplace = True)
df = df.sort_values('EventDate', ignore_index = True)

In [32]:
df.head()

,NtsbNo,Mkey,EventDate,City,State,Country,N#,HasSafetyRec,HighestInjuryLevel,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,FAR Part,Failure_Category,Region,SecondaryCause
0,ANC08MA038,67341,2008-01-05T13:43:00Z,Kodiak,Alaska,United States,N509FN,False,Fatal,6.0,3.0,1.0,135,Aircraft,Pacific West,Air_Maintenance Issue
1,CHI08LA071,67425,2008-01-09T07:49:00Z,Detroit,Michigan,United States,N349NB,False,None,0.0,0.0,0.0,121,Personnel issues,Midwest,Airport/Airline Personnel
2,SEA08LA061,67369,2008-01-13T19:29:00Z,San Francisco,California,United States,"N705SK, N508UA",False,None,0.0,0.0,0.0,121,Environmental issues,Pacific West,Env_Lighting/Visibility
3,SEA08FA062,67390,2008-01-14T05:08:00Z,Lihue,Hawaii,United States,N410UB,False,Fatal,1.0,0.0,0.0,135,Personnel issues,Pacific West,Pilot
4,ANC08LA031,67395,2008-01-15T14:39:00Z,Bethel,Alaska,United States,N454SF,False,Minor,0.0,0.0,1.0,135,Environmental issues,Pacific West,Env_Runway


**Creating Dummy Variables**

In [33]:
region_dummies = pd.get_dummies(df['Region'])
region_dummies.head()

,Caribbean,Midwest,Northeast,Other,Pacific,Pacific West,Plains,Southeast
0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0


In [39]:
FAR_dummies = pd.get_dummies(df['FAR Part'])
FAR_dummies.rename(columns = {121:'FAR_121', 135: 'FAR_135'}, inplace = True)
FAR_dummies.head()

,FAR_121,FAR_135
0,0,1
1,1,0
2,1,0
3,0,1
4,0,1


In [35]:
failure_dummies = pd.get_dummies(df['Failure_Category'])
failure_dummies.head()

,Aircraft,Environmental issues,Not determined,Organizational issues,Personnel issues
0,1,0,0,0,0
1,0,0,0,0,1
2,0,1,0,0,0
3,0,0,0,0,1
4,0,1,0,0,0


In [36]:
Sec_cause_dummies = pd.get_dummies(df['SecondaryCause'])
Sec_cause_dummies.head()

,Air_Damaged Part,Air_Failure,Air_Maintenance Issue,Air_Other,Aircraft Oper/Perf,Airport/Airline Personnel,Crew,Env_Equipment,Env_Lighting/Visibility,Env_Object/Animal,Env_Other,Env_Runway,Env_Turbulence,Env_Wind/Weather,Ground Crew,Not Determined,Org Issues,Passenger,Per_Other,Pilot
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [40]:
#Now to bring all of these dummie variables together
df_dummies = pd.concat([df, FAR_dummies, region_dummies, failure_dummies, Sec_cause_dummies], axis = 1)
df_dummies.head()

,NtsbNo,Mkey,EventDate,City,State,Country,N#,HasSafetyRec,HighestInjuryLevel,FatalInjuryCount,...,Env_Other,Env_Runway,Env_Turbulence,Env_Wind/Weather,Ground Crew,Not Determined,Org Issues,Passenger,Per_Other,Pilot
0,ANC08MA038,67341,2008-01-05T13:43:00Z,Kodiak,Alaska,United States,N509FN,False,Fatal,6.0,...,0,0,0,0,0,0,0,0,0,0
1,CHI08LA071,67425,2008-01-09T07:49:00Z,Detroit,Michigan,United States,N349NB,False,None,0.0,...,0,0,0,0,0,0,0,0,0,0
2,SEA08LA061,67369,2008-01-13T19:29:00Z,San Francisco,California,United States,"N705SK, N508UA",False,None,0.0,...,0,0,0,0,0,0,0,0,0,0
3,SEA08FA062,67390,2008-01-14T05:08:00Z,Lihue,Hawaii,United States,N410UB,False,Fatal,1.0,...,0,0,0,0,0,0,0,0,0,1
4,ANC08LA031,67395,2008-01-15T14:39:00Z,Bethel,Alaska,United States,N454SF,False,Minor,0.0,...,0,1,0,0,0,0,0,0,0,0


In [41]:
df_dummies.describe()

,Mkey,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,FAR Part,FAR_121,FAR_135,Caribbean,Midwest,Northeast,...,Env_Other,Env_Runway,Env_Turbulence,Env_Wind/Weather,Ground Crew,Not Determined,Org Issues,Passenger,Per_Other,Pilot
count,736.000000,735.000000,735.000000,735.000000,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000,...,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000,736.000000
mean,87121.326087,0.340136,0.421769,0.843537,129.331522,0.404891,0.595109,0.017663,0.129076,0.134511,...,0.008152,0.016304,0.146739,0.021739,0.039402,0.023098,0.005435,0.013587,0.008152,0.269022
std,10329.197068,2.116165,0.883740,5.266599,6.876867,0.491205,0.491205,0.131813,0.335512,0.341432,...,0.089982,0.126729,0.354086,0.145930,0.194682,0.150316,0.073570,0.115847,0.089982,0.443753
min,67341.000000,0.000000,0.000000,0.000000,121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,78100.250000,0.000000,0.000000,0.000000,121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,88886.000000,0.000000,0.000000,0.000000,135.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,95645.500000,0.000000,1.000000,0.000000,135.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,104343.000000,50.000000,9.000000,125.000000,135.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Split into Train/Test Datasets**

In [42]:
#Injury counts are going to be our dependent variable, since that is what will tell us which accident causes need to be addressed the most
X = df_dummies.drop(['FatalInjuryCount', 'SeriousInjuryCount', 'MinorInjuryCount'], axis = 1).values
y = df_dummies[['FatalInjuryCount', 'SeriousInjuryCount', 'MinorInjuryCount']].values

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

**Scaling the Injury Count Columns**

The three injury count columns are the only number columns that are not on a scale, going to fix that by creating a scaler and fitting and transforming the y_train data with it, then just transforming the test data.

In [46]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
y_train_sc = scaler.fit_transform(y_train)
y_test_sc = scaler.transform(y_test)

Next Steps: need to ask about saving dataframes that are split into train/test sections. Then Modeling is the next step.